In [14]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='./Config/.env', verbose=True)

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

import openai

openai.api_key = OPENAI_API_KEY

In [12]:
import openai
import json
from duckduckgo_search import DDGS
import ast


# DuckDuckGo에서 검색을 수행하는 함수를 정의
def duck_search(query) -> str:
    results = DDGS().text(query, max_results=5)
    return str(results)

In [13]:
duck_search('서울 인구수')

"[{'title': '서울특별시/인구 - 나무위키', 'href': 'https://namu.wiki/w/서울특별시/인구', 'body': '이후 1988년 서울올림픽이 열리는 해를 기점으로 마침내 1천만명을 돌파하게 되었다. 서울의 인구 분산을 위해 정부는 1989년 경기도 에 위성도시 로 1기 신도시 를 건설하면서 인구 유출의 조짐이 보이기 시작했다.'}, {'title': '인구로보는대한민국 - Kosis 국가통계포털', 'href': 'https://kosis.kr/visual/populationKorea/PopulationDashBoardMain.do', 'body': '통계청의 인구상황판은 전국, 지역별, 연령별, 성별별 인구 구조와 변화를 시각화한 그래프와 표를 제공합니다. 인구피라미드, 유소년, 생산연령, 고령, 노령화지수 등 다양한 인구 관련 정보를 쉽게 확인하고 공유할 수 있습니다.'}, {'title': '서울특별시 - 나무위키', 'href': 'https://namu.wiki/w/서울특별시', 'body': '서울특별시는 대한민국 수도 [21]로 최대도시이다. 평양시, 경주시, 개성시, 공주시, 전주시, 제주시와 함께 오랜 역사를 가진 한국의 고도 중 하나다. 법률상 대한민국 제1의 도시로 규정되어 있다. [22] 현재 대한민국에서 유일한 특별시고, 수장인 서울특별시장은 대한민국 광역자치단체장 중 ...'}, {'title': '주민등록 인구통계 행정안전부', 'href': 'https://jumin.mois.go.kr/', 'body': '행정안전부에서 제공하는 주민등록 인구통계는 행정기관별로 월간, 연간 인구수, 세대수, 세대당 인구, 남여 비율 등의 정보를 제공합니다. 전국, 광역시, 도, 특별자치도별로 통계표, 지도, 그래프,'}, {'title': '서울특별시 - 위키백과, 우리 모두의 백과사전', 'href': 'https://ko.wikipedia.org/wiki/서울특별시', 'body': '서울특별시(서울特別市, 영어:

In [ ]:
# OpenAI API를 호출하여 초기 응답
completion = openai.OpenAI().chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        functions=[
            {
                "name": "duck_search",
                "description": "Used to search online",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Translate the Korean content into English input query",
                        },
                    },
                    "required": ["query"],
                },
            }
        ],
        messages=[
            {"role": "system", 
            "content": "You must use the `duck_search` function to get information."},
            {"role": "user", 
            "content": "조지 클루니 생일이 언제야?"},
        ]
    )

In [27]:
import json

In [28]:

result = json.loads(completion.choices[0].message.function_call.arguments)

In [34]:
completion.choices[0].message.function_call

FunctionCall(arguments='{"query":"George Clooney birthday"}', name='duck_search')

In [31]:
search_result=duck_search(result.get('query'))

In [35]:
second_response = openai.OpenAI().chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", 
            "content": "You must use the `duck_search` function to get information."},
            {"role": "user", 
            "content": "조지 클루니 생일이 언제야?"},
            completion.choices[0].message,
            {
                "role": "function",
                "name": completion.choices[0].message.function_call.name,
                "content": search_result,
            },
        ],
    )

In [37]:
second_response.choices[0].message.content

'조지 클루니는 1961년 5월 6일에 태어났습니다.'